In [623]:
#!pip install requests jgraph tracery

In [624]:
import requests
# this downloads/requests stuff from the internet 

import csv
# this helps you parse csv

import re
# this uses regular expressions, or regex, a way to search grab chunks of text from a string

from pprint import pprint
# this helps us 'pretty print' data -- 'pretty print' is a term that means nicely formatting & outputting data

import random
# this helps us choose items from a list randomly, or to create random numbers, etc

import jgraph
# this helps us visualize the graph!

import tracery
# this is a library for generative text

# Poem Generator

### Download the data!
Replace `gsheet_url` with a link to a publicly viable Google Sheet

In [670]:
gsheet_url = "https://docs.google.com/spreadsheets/d/1RzZ8jivpLRQtFnCD39Rp3kzCRr_JnpyBkOAzLpVRIMw/edit#gid=443195507"

This cell uses RegEx, or regular expressions, to do an advanced search and extract the document ID and sheet ID from `gsheet_url`.

In [671]:
# define functions that extract the doc ID / sheet ID from gsheet_url
def getDocIdFromUrl(url):
    return re.search("([-\w]{25,})", url).group(1)
def getSheetIdFromUrl(url):
    return re.search("gid=([-\w]+)", url).group(1)

# craft the URL that gives us a CSV
gsheet_csv_url = "https://docs.google.com/spreadsheets/d/" + getDocIdFromUrl(gsheet_url) + "/export?gid=" + getSheetIdFromUrl(gsheet_url) + "&format=csv"

# if you're curious, try going to this url in your browser! It will automatically download a CSV file
print(gsheet_csv_url)

https://docs.google.com/spreadsheets/d/1RzZ8jivpLRQtFnCD39Rp3kzCRr_JnpyBkOAzLpVRIMw/export?gid=443195507&format=csv


Now we download the data! 
Optionally, if you want to get the data from a `*.csv` file that's saved on your computer, look at the next block.

In [672]:
# get response from URL
response = requests.get(gsheet_csv_url)

# make sure it's read/decoded with the right text encoding!
decoded_content = response.content.decode('utf-8')

# using the CSV library, parse the data! 
# Split the content on each 'newline', then make sure we use a comma as a delimiter to understand the difference between cells.
# also - ignore the first line, because it's the header info for the csv
edges = list(csv.reader(decoded_content.splitlines()[1:], delimiter=','))

print(edges)

[['code', 'societies']]


If you want to open a local CSV file on your computer, you can use the code in this comment! Just make sure that `sheet.csv` is in the same directory as this Jupyter notebook file.

In [673]:
"""
with open("sheet.csv", "r") as f:
    reader = csv.reader(f)
    edges = list(reader)

print(edges)
"""

'\nwith open("sheet.csv", "r") as f:\n    reader = csv.reader(f)\n    edges = list(reader)\n\nprint(edges)\n'

In [674]:

jgraph.draw(edges)

### Network Time


In [675]:
all_nodes = []

for e in edges:
    # add the first node in the edge to all_nodes
    all_nodes.append(e[0])
    # add the second node in the edge to all_nodes
    all_nodes.append(e[1])
    
# a 'set' is a data structure that can only contain unique elements --
# so by converting to a set and back to a list, we get only a unique list of nodes.
unique_nodes = list(set(all_nodes))


#### here's a challenge: the single line below does the exact same thing as the lines of code above. can you see why?
# unique_nodes = list(set([e for edge in edges for e in edge]))

In [676]:
# let's create a graph! 
# let's define a graph as a dict, which stores things in a key-value structure
# for dicts, keys are like names on mailboxes, values are like the content inside the mailbox
# you can 'look up' data using the key, and get what's inside.
# dicts are great way to structure information.

graph = {}

# iterate through edges

for edge in edges:
    
    # for each edge, let's define 'node_from' and 'node_to' so that the code is easier for humans to read

    node_from = edge[0]
    node_to = edge[1]
    
    # if the node_from doesn't already exist in the 'graph' dict, 
    # let's insert node_from as a key, and have the value be an empty array

    if(node_from not in graph):
        graph[node_from] = []
    
    # in the array that exists in graph[node_from], add node_to

    graph[node_from].append(node_to)
    
    # let's do it in the reverse, since we'll assume that the graph isn't directed
    # which means that there's no directional relationship.
 
    if(node_to not in graph):
        graph[node_to] = []
    graph[node_to].append(node_from)
    
# now you can see how the data is structured!
# compare this against the data output from the CSV and see if you can tell what's going on intuitively.
pprint(graph)

{'code': ['societies'], 'societies': ['code']}


In [677]:
# let's see all the keys!
# this happens to equal the list of 'unique_nodes'.. and that's not a coincidence
print(graph.keys())


dict_keys(['code', 'societies'])


In [678]:
# choose a random node and hop skip randomly to what it's connected to

# choose a random node!
current_node = random.choice(list(graph.keys()))

print (thisnode + " ==> ")

# do this five times
for i in range(5):
    
    # find all the options we can go to in the graph from 'current_node'
    to_options = graph[current_node]
    # pick one of those randomly and set 'current_node' that node
    current_node = random.choice(to_options)
    
    print (current_node + " ==> ")
    
    
# try running this repeatedly

work exchange ==> 
code ==> 
societies ==> 
code ==> 
societies ==> 
code ==> 


In [679]:
# let's put this inside a function!

def randomly_traverse_graph(graph, number_of_steps):

    current_node = random.choice(list(graph.keys()))
    
    steps = []
    steps.append(current_node)
    
    for i in range(number_of_steps - 1):
        to_options = graph[current_node]
        current_node = random.choice(to_options)
        steps.append(current_node)
        
    return steps
    
    
    
print(randomly_traverse_graph(graph, 5))


['code', 'societies', 'code', 'societies', 'code']


In [680]:
list_of_verbs = ["affects", "changes", "alters", "relates to", "thinks about","touches","creates dreams about","helps","also is connected to"]


In [681]:
steps = randomly_traverse_graph(graph, 5)

for s in steps:
    random_verb = random.choice(list_of_verbs)
    print(s + " " + random_verb)
    

code touches
societies creates dreams about
code relates to
societies changes
code creates dreams about


In [682]:
poem = ""
for i in range(len(steps) - 1):
    step = steps[i]
    next_step = steps[i + 1]
    poem += step + " " + random.choice(list_of_verbs) + " " + next_step
    
    if(i == len(steps) - 2):
        poem += "."
    else:
        poem += "; "    


print(poem)

code helps societies; societies creates dreams about code; code helps societies; societies touches code.


In [683]:
from tracery.modifiers import base_english


In [684]:
def generate_simple_line(node_from, node_to):
    grammarSource = {
        "node_from": node_from,
        "node_to": node_to,
        "adverb": "suddenly|silently|happily|surprisingly|calmly|beautifully".split("|"),
        "verb":  ["affects", "dances around", "changes", "alters", "relates to", "thinks about","touches","creates dreams about","helps","also is connected to"],
        "poem": "#node_from# #adverb# #verb# #node_to#"
    }
    grammar = tracery.Grammar(grammarSource)
    return grammar.flatten("#poem#")
    
print(generate_simple_line("the moon", "the sun"))


the moon suddenly helps the sun


In [685]:
def generate_poem_line(node_from, node_to):
    grammarSource = {
        "node_from": node_from,
        "node_to": node_to,
        "wonderadj": "quiet|solemn|contemplative|still|tender|harmonious".split("|"),
        "tensionadj": "trembling|buzzing|vibrating|swirling".split("|"),
        "adj": "#wonderadj#,|#tensionadj#,".split("|"),
        "name": "fire|bother|wonder|winter".split("|"),
        "earlylate": "early |late |mid-".split("|"),
        "season": "autumn|summer|fall|winter".split("|"),
        "seasonornot": "|in #earlylate##season#, ".split("|"),
        "adverbornot": "|#adverb#".split("|"),
        "adverb": "briskly|tartly|simply|harshly|fuzzily|freely|ably|copiously|furtively|endlessly|sarcastically|generatively|slowly|distinctly".split("|"),
        "verbs": "dances|thinks|touches|dreams|hesitates|loves|feels|wanders|does|sits|reads|looks|runs|asks|collapses|chats|donates".split("|"),
        "preposition": "around|about|with|of|to|from".split("|"),
        "lineending": ['',',',';','--','.'],
        "poemline1": "#adj# #node_from# #adverb# #verbs# #preposition# #node_to##lineending#",
        "poemline2": "#seasonornot##node_from# and #node_to# #adverb# #verbs##lineending#",
        "poem": "#poemline1#|#poemline2#".split("|")
    }
    grammar = tracery.Grammar(grammarSource)
    return grammar.flatten("#poem#")
    
print(generate_poem_line("moon", "sun"))


trembling, moon simply chats from sun.


In [686]:
steps = randomly_traverse_graph(graph, 10)

poem = ""
for i in range(len(steps) - 1):
    step = steps[i]
    next_step = steps[i + 1]
    poem += generate_poem_line(step, next_step) + "\n"

print(poem)

in early fall, code and societies briskly thinks;
buzzing, societies fuzzily donates about code--
vibrating, code copiously looks about societies
harmonious, societies distinctly loves to code--
swirling, code fuzzily dreams about societies.
harmonious, societies slowly does with code.
trembling, code copiously collapses of societies
societies and code tartly dreams;
code and societies freely donates;

